<a href="https://colab.research.google.com/github/Aqqad2004/ClientServerSystem/blob/main/GRS_LLM_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bitsandbytes accelerate
!pip install nbformat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [4]:
import nbformat

# load the broken notebook
nb = nbformat.read("/content/Starting_Code_Thesis (3).ipynb", as_version=nbformat.NO_CONVERT)

# remove any `metadata.widgets` entries
for cell in nb.cells:
    if "widgets" in cell.metadata:
        del cell.metadata["widgets"]

# write out a cleaned copy (you can overwrite the old one or write a new file)
nbformat.write(nb, "Starting_Code_Thesis_clean.ipynb")


In [ ]:
import torch
import pandas as pd
import numpy as np
import json
import re
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Initialize the Accelerator (useful for multi-device setups)
accelerator = Accelerator()

# -----------------------------------------------------------------------------
# 1. CLASS DEFINITIONS & PROMPT SETUP
# -----------------------------------------------------------------------------

class Recommendation(BaseModel):
    strategy: str = Field(description="The aggregation strategy that was applied")
    recommendation: list = Field(description="List of exactly 10 recommended movieIds")

# Create a JSON parser for the output schema
parser = JsonOutputParser(pydantic_object=Recommendation)

# Note: The prompt now uses a placeholder {strat} for the strategy instruction.
prompt = PromptTemplate(
    template="""Only reply with a JSON object strictly in the following format (do not include markdown or code fences):
{{
  "recommendation": [movieId1, movieId2, movieId3, movieId4, movieId5, movieId6, movieId7, movieId8, movieId9, movieId10],
  "strategy": "the strategy that was applied"
}}

You are an expert in making group recommendations based on a table of movie ratings.
The table below includes the ratings (on a scale from 0 to 5) provided by each user for various movies.
You are to apply the following aggregation strategy:

## Aggregation Strategy ##
{strat}
## End Aggregation Strategy ##

Based on the group table below, provide exactly 10 recommended movieIds that are present in the table.
If multiple items share the same score, list all of them (ensure the final recommendation always contains exactly 10 items).
If no recommendation is possible, return an empty list.

## Group Table ##
{desc}
## End Group Table ##
""",
    input_variables=["desc", "strat"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# -----------------------------------------------------------------------------
# 2. HUGGING FACE BACKEND SETUP
# -----------------------------------------------------------------------------

global_model = None
global_tokenizer = None

def get_model_and_tokenizer():
    global global_model, global_tokenizer
    if global_model is None or global_tokenizer is None:
        model_name = "google/gemma-3-1b-it"  # Change this if you wish to experiment with other models
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)
        import os
        token = os.environ.get("HF_TOKEN")  # set your Hugging Face token as an environment variable, if needed

        print("Loading model with BitsAndBytes 8bit quantization...")
        global_tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_auth_token=token,
            trust_remote_code=True
        )
        device = accelerator.device
        global_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            use_auth_token=token,
            trust_remote_code=True
        )
        print("Using device:", str(device).upper())
        global_model.eval()
        global_tokenizer.pad_token = global_tokenizer.eos_token
    return global_model, global_tokenizer

# -----------------------------------------------------------------------------
# 3. LLM-BASED RECOMMENDATION FUNCTION
# -----------------------------------------------------------------------------

def query_llm_for_recommendations(user_ids, train_df, strategy_text, model_type='huggingface', force_cpu=False):
    # Prepare group data by filtering for the selected users
    group_df = train_df[train_df['userId'].isin(user_ids)].copy()

    # Limit the number of movies if there are too many, to avoid huge prompts
    if group_df['movieId'].nunique() > 500:
        sampled_movies = np.random.choice(group_df['movieId'].unique(), 500, replace=False)
        group_df = group_df[group_df['movieId'].isin(sampled_movies)]

    # Pivot the data so that each user is a row and columns represent movie ratings
    group_pivot = group_df.pivot(index="userId", columns="movieId", values="rating")
    group_pivot.reset_index(inplace=True)
    group_desc_str = json.dumps(group_pivot.to_dict(orient="list"), indent=2)

    full_prompt = prompt.format(desc=group_desc_str, strat=strategy_text)
    recommendations = ""

    if model_type == 'huggingface':
        model, tokenizer = get_model_and_tokenizer()
        llm_pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            do_sample=False,
            temperature=0.0,
        )

        generated = llm_pipe(
            full_prompt,
            max_new_tokens=1000,
            return_full_text=False,
        )
        recommendations = generated[0]["generated_text"]
    else:
        raise ValueError("Invalid model type. Choose from 'huggingface'.")

    print("Raw LLM output:\n", recommendations)
    return recommendations

# -----------------------------------------------------------------------------
# 4. DATA LOADING & PREPROCESSING FUNCTIONS
# -----------------------------------------------------------------------------

def load_movielens_data(ratings_file):
    df = pd.read_csv('/content/ratings.csv')
    df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)
    return df

def train_test_split_by_user(df, test_ratio=0.2, seed=42):
    np.random.seed(seed)
    train_list, test_list = [], []
    for user, group in df.groupby("userId"):
        if len(group) < 2:
            train_list.append(group)
            continue
        test_count = max(1, int(len(group) * test_ratio))
        test_sample = group.sample(n=test_count, random_state=seed)
        train_sample = group.drop(test_sample.index)
        train_list.append(train_sample)
        test_list.append(test_sample)
    return pd.concat(train_list), pd.concat(test_list)

# -----------------------------------------------------------------------------
# 5. TRADITIONAL AGGREGATION STRATEGIES (BASELINES)
# -----------------------------------------------------------------------------

def add_aggregation(user_ids, train_df, top_n=10):
    candidate_items = set()
    for uid in user_ids:
        candidate_items.update(train_df[train_df['userId'] == uid]['movieId'].unique())
    add_scores = {}
    for movie in candidate_items:
        ratings = train_df[(train_df['movieId'] == movie) & (train_df['userId'].isin(user_ids))]['rating']
        if not ratings.empty:
            add_scores[movie] = ratings.sum()
    sorted_movies = sorted(add_scores, key=add_scores.get, reverse=True)
    return sorted_movies[:top_n]

def app_aggregation(user_ids, train_df, top_n=10, threshold=3):
    # Adapted threshold: since ratings are on a scale from 0 to 5.
    candidate_items = set()
    for uid in user_ids:
        candidate_items.update(train_df[train_df['userId'] == uid]['movieId'].unique())
    app_scores = {}
    for movie in candidate_items:
        ratings = train_df[(train_df['movieId'] == movie) & (train_df['userId'].isin(user_ids))]['rating']
        if not ratings.empty:
            app_scores[movie] = (ratings > threshold).sum()
    sorted_movies = sorted(app_scores, key=app_scores.get, reverse=True)
    return sorted_movies[:top_n]

def LMS(train_df, group_id=1, top_n=10):
    # Use the group's min ratings (least misery) aggregation
    counts = train_df.groupby(["groupId", "movieId"])["rating"].min().reset_index(name="min_rating")
    return list(counts.loc[counts["min_rating"] == counts["min_rating"].max()]["movieId"].head(10))

def mpl_aggregation(user_ids, train_df, top_n=10):
    candidate_items = set()
    for uid in user_ids:
        candidate_items.update(train_df[train_df['userId'] == uid]['movieId'].unique())
    mpl_scores = {}
    for movie in candidate_items:
        ratings = train_df[(train_df['movieId'] == movie) & (train_df['userId'].isin(user_ids))]['rating']
        if not ratings.empty:
            mpl_scores[movie] = ratings.max()
    sorted_movies = sorted(mpl_scores, key=mpl_scores.get, reverse=True)
    return sorted_movies[:top_n]

# Dictionary mapping for aggregation strategies
aggregation_strategies = {
    "ADD": add_aggregation,
    "APP": app_aggregation,
    "LMS": lambda uids, df, top_n=10: LMS(df, group_id=1, top_n=top_n),  # wrapper to match parameters
    "MPL": mpl_aggregation,
}

# Strategy instruction texts for the LLM prompt
strategy_texts = {
    "ADD": "ADD: ADD sums all ratings per item and recommends the item with the highest sum (Senot et al. 2010).",
    "APP": "APP: APP is a majority-based strategy. For each item, count the number of ratings above 3. Use APP to refer to this strategy.",
    "LMS": "LMS: LMS recommends the item with the highest rating when considering only each item’s lowest rating (Senot et al. 2010).",
    "MPL": "MPL: MPL recommends the item with the highest single rating across individuals (Senot et al. 2010).",
}

# -----------------------------------------------------------------------------
# 6. EVALUATION METRICS
# -----------------------------------------------------------------------------

def precision_at_n(recommended_list, baseline_list, top_n=10):
    hits = [rec for rec in recommended_list[:top_n] if rec in baseline_list]
    return len(hits) / top_n

def ndcg_at_n(recommended_list, baseline_list, top_n=10):
    relevance_scores = [1 if item in baseline_list else 0 for item in recommended_list[:top_n]]
    dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores))
    idcg = sum(1 / np.log2(idx + 2) for idx in range(min(len(baseline_list), top_n)))
    return dcg / idcg if idcg > 0 else 0

# -----------------------------------------------------------------------------
# 7. EXPERIMENTS: VARYING GROUP AND MOVIE SAMPLE SIZES
# -----------------------------------------------------------------------------

def run_experiments():
    # Update the path to your ratings file as needed:
    ratings_file = 'C:/Users/moala/OneDrive/Documents/UNIVERSITY WORK/Year 3/THESIS/ml-32m/ml-32m/ratings.csv'
    df = load_movielens_data(ratings_file)
    train_df, _ = train_test_split_by_user(df)

    unique_users = train_df["userId"].unique()
    group_sizes = [2, 4, 6, 8]
    movie_sample_sizes = [50, 100, 500]

    for group_size in group_sizes:
        # Randomly sample group_size users from unique users
        group_user_ids = np.random.choice(unique_users, group_size, replace=False)
        group_df = train_df[train_df["userId"].isin(group_user_ids)].copy()

        for sample_size in movie_sample_sizes:
            # If there are more movies than sample_size in the group, sample a subset
            unique_movies = group_df["movieId"].unique()
            if len(unique_movies) > sample_size:
                sampled_movies = np.random.choice(unique_movies, sample_size, replace=False)
                current_group_df = group_df[group_df["movieId"].isin(sampled_movies)]
            else:
                current_group_df = group_df.copy()

            # Add a group identifier (used by the LMS function)
            current_group_df["groupId"] = 1

            print(f"\n--- Experiment: Group size = {group_size}, Movie sample size = {sample_size} ---")

            for strat in aggregation_strategies:
                # Compute the baseline recommendation using the given aggregation strategy
                baseline_func = aggregation_strategies[strat]
                baseline_recs = baseline_func(group_user_ids, current_group_df, top_n=10)
                print(f"\nBaseline recommendation using {strat} strategy: {baseline_recs}")

                # Query the LLM with the corresponding strategy text.
                strategy_instruction = strategy_texts[strat]
                print("Querying LLM for recommendation...")
                llm_output = query_llm_for_recommendations(group_user_ids, current_group_df, strategy_text=strategy_instruction)

                # Clean the output by removing any markdown formatting if present.
                llm_output_clean = llm_output.strip()
                llm_output_clean = re.sub(r"^```[a-z]*", "", llm_output_clean)
                llm_output_clean = re.sub(r"```$", "", llm_output_clean).strip()

                try:
                    parsed_output = parser.parse(llm_output_clean)
                    if isinstance(parsed_output, dict):
                        llm_recommendations = parsed_output.get("recommendation", [])
                    else:
                        llm_recommendations = parsed_output.recommendation

                    if len(llm_recommendations) > 10:
                        llm_recommendations = llm_recommendations[:10]
                    try:
                        llm_recommendations = [int(x) for x in llm_recommendations]
                    except Exception as e:
                        print("Could not convert recommendations to integers:", e)
                except Exception as e:
                    print("Error parsing LLM output:", e)
                    llm_recommendations = []

                print(f"LLM-based recommendation using {strat} strategy: {llm_recommendations}")

                # Compute and print the evaluation metrics comparing the LLM recommendations to the baseline.
                prec = precision_at_n(llm_recommendations, baseline_recs, top_n=10)
                ndcg_val = ndcg_at_n(llm_recommendations, baseline_recs, top_n=10)
                print(f"Evaluation Metrics for {strat} strategy:\n  Precision@10: {prec:.2f}\n  nDCG@10: {ndcg_val:.2f}")

# -----------------------------------------------------------------------------
# 8. MAIN EXECUTION
# -----------------------------------------------------------------------------

if __name__ == '__main__':
    run_experiments()



--- Experiment: Group size = 2, Movie sample size = 50 ---

Baseline recommendation using ADD strategy: [np.int64(92422), np.int64(4316), np.int64(277), np.int64(94780), np.int64(58559), np.int64(2087), np.int64(81591), np.int64(1391)]
Querying LLM for recommendation...
Loading model with BitsAndBytes 8bit quantization...


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


Using device: CUDA


...

Raw LLM output:
 ```json
{
  "recommendation": [
    "94780",
    "92422",
    "81591",
    "6369",
    "117432",
    "58559",
    "4316",
    "277",
    "1391"
  ],
  "strategy": "ADD"
}
```

LLM-based recommendation using ADD strategy: [94780, 92422, 81591, 6369, 117432, 58559, 4316, 277, 1391]
Evaluation Metrics for ADD strategy:
  Precision@10: 0.70
  nDCG@10: 0.87

Baseline recommendation using APP strategy: [np.int64(92422), np.int64(2087), np.int64(4316), np.int64(277), np.int64(94780), np.int64(58559), np.int64(1391), np.int64(81591)]
Querying LLM for recommendation...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Device set to use cuda:0


Raw LLM output:
 ```json
{
  "recommendation": [
    "94780",
    "92422",
    "81591",
    "1391",
    "6369",
    "117432",
    "58559",
    "4316",
    "277"
  ],
  "strategy": "APP"
}
```

LLM-based recommendation using APP strategy: [94780, 92422, 81591, 1391, 6369, 117432, 58559, 4316, 277]
Evaluation Metrics for APP strategy:
  Precision@10: 0.70
  nDCG@10: 0.89

Baseline recommendation using LMS strategy: [277, 4316, 58559, 92422, 94780]
Querying LLM for recommendation...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


KeyboardInterrupt: 